In [1]:
import carla
import time
import os

In [2]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

vehicle_bp = blueprint_library.filter("vehicle.tesla.model3")[0]
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

In [3]:
# LiDAR sensor
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '50')
lidar_location = carla.Location(0, 0, 2)
lidar_rotation = carla.Rotation(0, 0, 0)
lidar_transform = carla.Transform(lidar_location, lidar_rotation)
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)

In [4]:
# RADAR sensor
radar_bp = blueprint_library.find('sensor.other.radar')
radar_location = carla.Location(0.2, 0, 1)
radar_rotation = carla.Rotation(0, 0, 0)
radar_transform = carla.Transform(radar_location, radar_rotation)
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=vehicle)

In [5]:
# Collision sensor
collision_bp = blueprint_library.find('sensor.other.collision')
collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)

In [6]:
# Saving data
def save_lidar_data(data):
    filename = f"output/lidar/{data.frame}.ply"
    data.save_to_disk(filename)

def save_radar_data(data):
    filename = f"output/radar/{data.frame}.radar"
    with open(filename, 'wb') as f:
        f.write(data.raw_data)

def save_collision_data(event):
    filename = f"output/collision/{event.frame}.txt"
    with open(filename, 'w') as f:
        f.write(str(event))

In [7]:
# Create Folders
os.makedirs('output/lidar', exist_ok=True)
os.makedirs('output/radar', exist_ok=True)
os.makedirs('output/collision', exist_ok=True)

In [8]:
lidar.listen(lambda data: save_lidar_data(data))
radar.listen(lambda data: save_radar_data(data))
collision_sensor.listen(lambda event: save_collision_data(event))

time.sleep(30)

In [9]:
# Cleaning
lidar.stop()
radar.stop()
collision_sensor.stop()
vehicle.destroy()
lidar.destroy()
radar.destroy()
collision_sensor.destroy()

True